In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI: Vertex AI Migration: AutoML Image Object Detection

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/UJ5 Vertex SDK AutoML Image Object Detection.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/UJ5 Vertex SDK AutoML Image Object Detection.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/UJ5 Vertex SDK AutoML Image Object Detection.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>
<br/><br/><br/>

### Dataset

The dataset used for this tutorial is the Salads category of the [OpenImages dataset](https://www.tensorflow.org/datasets/catalog/open_images_v4) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). This dataset does not require any feature engineering. The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket. The trained model predicts the bounding box locations and corresponding type of salad items in an image from a class of five items: salad, seafood, tomato, baked goods, or cheese.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

If you are using Colab or Google Cloud Notebooks, your environment already meets all the requirements to run this notebook. You can skip this step.

Otherwise, make sure your environment meets this notebook's requirements. You need the following:

- The Cloud Storage SDK
- Git
- Python 3
- virtualenv
- Jupyter notebook running in a virtual environment with Python 3

The Cloud Storage guide to [Setting up a Python development environment](https://cloud.google.com/python/setup) and the [Jupyter installation guide](https://jupyter.org/install) provide detailed instructions for meeting these requirements. The following steps provide a condensed set of instructions:

1. [Install and initialize the SDK](https://cloud.google.com/sdk/docs/).

2. [Install Python 3](https://cloud.google.com/python/setup#installing_python).

3. [Install virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) and create a virtual environment that uses Python 3.  Activate the virtual environment.

4. To install Jupyter, run `pip3 install jupyter` on the command-line in a terminal shell.

5. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

6. Open this notebook in the Jupyter Notebook Dashboard.


## Installation

Install the latest version of Vertex SDK for Python.

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Install the latest GA version of *google-cloud-storage* library as well.

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
! pip3 install --upgrade tensorflow $USER_FLAG

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [1]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Before you begin

### GPU runtime

This tutorial does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the following APIs: Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. If you are running this notebook locally, you will need to install the [Cloud SDK]((https://cloud.google.com/sdk)).

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

**Click Create service account**.

In the **Service account name** field, enter a name, and click **Create**.

In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex" into the filter box, and select **Vertex Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

Click Create. A JSON file that contains your key downloads to your local environment.

Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
import google.cloud.aiplatform as aip

## Initialize Vertex SDK for Python

Initialize the Vertex SDK for Python for your project and corresponding bucket.

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

#### Location of Cloud Storage training data.

Now set the variable `IMPORT_FILE` to the location of the CSV index file in Cloud Storage.

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/vision/salads.csv"

#### Quick peek at your data

This tutorial uses a version of the Salads dataset that is stored in a public Cloud Storage bucket, using a CSV index file.

Start by doing a quick peek at the data. You count the number of examples by counting the number of rows in the CSV index file  (`wc -l`) and then peek at the first few rows.

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

## Create a dataset

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

### Create the Dataset

Next, create the `Dataset` resource using the `create` method for the `ImageDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.
- `import_schema_uri`: The data labeling schema for the data items.

This operation may take several minutes.

In [ ]:
dataset = aip.ImageDataset.create(
    display_name="Salads" + "_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.image.bounding_box,
)

print(dataset.resource_name)

*Example Output:*

    INFO:google.cloud.aiplatform.datasets.dataset:Creating ImageDataset
    INFO:google.cloud.aiplatform.datasets.dataset:Create ImageDataset backing LRO: projects/759209241365/locations/us-central1/datasets/2940964905882222592/operations/1941426647739662336
    INFO:google.cloud.aiplatform.datasets.dataset:ImageDataset created. Resource name: projects/759209241365/locations/us-central1/datasets/2940964905882222592
    INFO:google.cloud.aiplatform.datasets.dataset:To use this ImageDataset in another session:
    INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.ImageDataset('projects/759209241365/locations/us-central1/datasets/2940964905882222592')
    INFO:google.cloud.aiplatform.datasets.dataset:Importing ImageDataset data: projects/759209241365/locations/us-central1/datasets/2940964905882222592
    INFO:google.cloud.aiplatform.datasets.dataset:Import ImageDataset data backing LRO: projects/759209241365/locations/us-central1/datasets/2940964905882222592/operations/8100099138168815616
    INFO:google.cloud.aiplatform.datasets.dataset:ImageDataset data imported. Resource name: projects/759209241365/locations/us-central1/datasets/2940964905882222592
    projects/759209241365/locations/us-central1/datasets/2940964905882222592

## Train a model

### [training.automl-api](https://cloud.google.com/vertex-ai/docs/training/automl-api)

### Create and run training pipeline

To train an AutoML model, you perform two steps: 1) create a training pipeline, and 2) run the pipeline.

#### Create training pipeline

An AutoML training pipeline is created with the `AutoMLImageTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `prediction_type`: The type task to train the model for.
  - `classification`: An image classification model.
  - `object_detection`: An image object detection model.
- `multi_label`: If a classification task, whether single (`False`) or multi-labeled (`True`).
- `model_type`: The type of model for deployment.
  - `CLOUD`: Deployment on Google Cloud
  - `CLOUD_HIGH_ACCURACY_1`: Optimized for accuracy over latency for deployment on Google Cloud.
  - `CLOUD_LOW_LATENCY_`: Optimized for latency over accuracy for deployment on Google Cloud.
  - `MOBILE_TF_VERSATILE_1`: Deployment on an edge device.
  - `MOBILE_TF_HIGH_ACCURACY_1`:Optimized for accuracy over latency for deployment on an edge device.
  - `MOBILE_TF_LOW_LATENCY_1`: Optimized for latency over accuracy for deployment on an edge device.
- `base_model`: (optional) Transfer learning from existing `Model` resource -- supported for image classification only.

The instantiated object is the DAG (directed acyclic graph) for the training job.

In [ ]:
dag = aip.AutoMLImageTrainingJob(
    display_name="salads_" + TIMESTAMP,
    prediction_type="object_detection",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

print(dag)

*Example output:*

    <google.cloud.aiplatform.training_jobs.AutoMLImageTrainingJob object at 0x7f806a6116d0>

#### Run the training pipeline

Next, you run the DAG to start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `validation_fraction_split`: The percentage of the dataset to use for validation.
- `budget_milli_node_hours`: (optional) Maximum training time specified in unit of millihours (1000 = hour).
- `disable_early_stopping`: If `True`, training maybe completed before using the entire budget if the service believes it cannot further improve on the model objective measurements.

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take upto 20 minutes.

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="salads_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=20000,
    disable_early_stopping=False,
)

*Example output:*

    INFO:google.cloud.aiplatform.training_jobs:View Training:
    https://console.cloud.google.com/ai/platform/locations/us-central1/training/2109316300865011712?project=759209241365
    INFO:google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/2109316300865011712 current state:
    PipelineState.PIPELINE_STATE_RUNNING
    INFO:google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/2109316300865011712 current state:
    PipelineState.PIPELINE_STATE_RUNNING
    INFO:google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/2109316300865011712 current state:
    PipelineState.PIPELINE_STATE_RUNNING
    INFO:google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/2109316300865011712 current state:
    PipelineState.PIPELINE_STATE_RUNNING
    INFO:google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/2109316300865011712 current state:
    PipelineState.PIPELINE_STATE_RUNNING
    ...
    INFO:google.cloud.aiplatform.training_jobs:AutoMLImageTrainingJob run completed. Resource name: projects/759209241365/locations/us-central1/trainingPipelines/2109316300865011712
    INFO:google.cloud.aiplatform.training_jobs:Model available at projects/759209241365/locations/us-central1/models/1284590221056278528

## Evaluate the model

### [projects.locations.models.evaluations.list](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

## Review model evaluation scores
After your model has finished training, you can review the evaluation scores for it.

First, you need to get a reference to the new model. As with datasets, you can either use the reference to the model variable you created when you deployed the model or you can list all of the models in your project.

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=salads_" + TIMESTAMP)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

*Example output:*

    name: "projects/759209241365/locations/us-central1/models/623915674158235648/evaluations/4280507618583117824"
    metrics_schema_uri: "gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml"
    metrics {
      struct_value {
        fields {
          key: "auPrc"
          value {
            number_value: 0.9891107
          }
        }
        fields {
          key: "confidenceMetrics"
          value {
            list_value {
              values {
                struct_value {
                  fields {
                    key: "precision"
                    value {
                      number_value: 0.2
                    }
                  }
                  fields {
                    key: "recall"
                    value {
                      number_value: 1.0
                    }
                  }
                }
              }

## Make batch predictions

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### Get test item(s)

Now do a batch prediction to your Vertex model. You will use arbitrary examples out of the dataset as a test items. Don't be concerned that the examples were likely used in training the model -- we just want to demonstrate how to make a prediction.

In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n2
cols_1 = str(test_items[0]).split(",")
cols_2 = str(test_items[1]).split(",")
if len(cols_1) == 11:
    test_item_1 = str(cols_1[1])
    test_label_1 = str(cols_1[2])
    test_item_2 = str(cols_2[1])
    test_label_2 = str(cols_2[2])
else:
    test_item_1 = str(cols_1[0])
    test_label_1 = str(cols_1[1])
    test_item_2 = str(cols_2[0])
    test_label_2 = str(cols_2[1])

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### Copy test item(s)

For the batch prediction, copy the test items over to your Cloud Storage bucket.

In [ ]:
file_1 = test_item_1.split("/")[-1]
file_2 = test_item_2.split("/")[-1]

! gsutil cp $test_item_1 $BUCKET_NAME/$file_1
! gsutil cp $test_item_2 $BUCKET_NAME/$file_2

test_item_1 = BUCKET_NAME + "/" + file_1
test_item_2 = BUCKET_NAME + "/" + file_2

### Make the batch input file

Now make a batch input file, which you will store in your local Cloud Storage bucket. The batch input file can be either CSV or JSONL. You will use JSONL in this tutorial. For JSONL file, you make one dictionary entry per line for each data item (instance). The dictionary contains the key/value pairs:

- `content`: The Cloud Storage path to the image.
- `mime_type`: The content type. In our example, it is a `jpeg` file.

For example:

                        {'content': '[your-bucket]/file1.jpg', 'mime_type': 'jpeg'}

In [ ]:
import json

import tensorflow as tf

gcs_input_uri = BUCKET_NAME + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": test_item_1, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")
    data = {"content": test_item_2, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### Make the batch prediction request

Now that your Model resource is trained, you can make a batch prediction by invoking the batch_predict() method, with the following parameters:

- `job_display_name`: The human readable name for the batch prediction job.
- `gcs_source`: A list of one or more batch request input files.
- `gcs_destination_prefix`: The Cloud Storage location for storing the batch prediction resuls.
- `sync`: If set to True, the call will block while waiting for the asynchronous batch job to complete.

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="salads_" + TIMESTAMP,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_NAME,
    sync=False,
)

print(batch_predict_job)

*Example output:*

    INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob
    <google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f806a6112d0> is waiting for upstream dependencies to complete.
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
    INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
    INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296')
    INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296 current state:
    JobState.JOB_STATE_RUNNING

### Wait for completion of batch prediction job

Next, wait for the batch job to complete. Alternatively, one can set the parameter `sync` to `True` in the `batch_predict()` method to block until the batch prediction job is completed.

In [ ]:
batch_predict_job.wait()

*Example Output:*

    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
    INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
    INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328')
    INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_SUCCEEDED
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob run completed. Resource name: projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

### Get the predictions

Next, get the results from the completed batch prediction job.

The results are written to the Cloud Storage output bucket you specified in the batch prediction request. You call the method iter_outputs() to get a list of each Cloud Storage file generated with the results. Each file contains one or more prediction requests in a JSON format:

- `content`: The prediction request.
- `prediction`: The prediction response.
 - `ids`: The internal assigned unique identifiers for each prediction request.
 - `displayNames`: The class names for each class label.
 - `bboxes`: The bounding box of each detected object.

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

*Example Output:*

    Prediction(predictions=[{'ids': ['225634079670796288', '2099131524656922624', '4404974533870616576', '6710817543084310528', '225634079670796288', '2099131524656922624', '6710817543084310528', '2099131524656922624', '225634079670796288', '6710817543084310528', '4404974533870616576', '6710817543084310528', '9016660552298004480', '2099131524656922624', '225634079670796288', '225634079670796288', '225634079670796288', '9016660552298004480', '4404974533870616576', '4404974533870616576', '2099131524656922624', '4404974533870616576', '4404974533870616576', '2099131524656922624', '9016660552298004480', '2099131524656922624', '4404974533870616576', '6710817543084310528', '2099131524656922624', '2099131524656922624', '2099131524656922624', '6710817543084310528', '6710817543084310528', '9016660552298004480', '4404974533870616576', '225634079670796288', '2099131524656922624', '2099131524656922624', '4404974533870616576', '225634079670796288'], 'confidences': [0.993109703, 0.985925615, 0.0453977883, 0.044005096, 0.0327100307, 0.0218445472, 0.019706985, 0.0119887572, 0.0119416527, 0.00427311379, 0.00346497213, 0.00306943036, 0.00277529657, 0.00233747251, 0.00226957048, 0.00169593049, 0.00143932423, 0.00143866683, 0.0012890877, 0.00107431656, 0.00104699621, 0.000975079951, 0.000959897472, 0.000695129682, 0.000612194242, 0.000498361886, 0.000362998835, 0.000331683928, 0.000291648932, 0.000270543911, 0.000256592786, 0.000191880303, 0.000177405163, 0.000124130107, 0.000117361531, 0.00010169336, 8.34302919e-05, 7.46671e-05, 7.43425626e-05, 7.28466039e-05], 'displayNames': ['Salad', 'Baked Goods', 'Cheese', 'Seafood', 'Salad', 'Baked Goods', 'Seafood', 'Baked Goods', 'Salad', 'Seafood', 'Cheese', 'Seafood', 'Tomato', 'Baked Goods', 'Salad', 'Salad', 'Salad', 'Tomato', 'Cheese', 'Cheese', 'Baked Goods', 'Cheese', 'Cheese', 'Baked Goods', 'Tomato', 'Baked Goods', 'Cheese', 'Seafood', 'Baked Goods', 'Baked Goods', 'Baked Goods', 'Seafood', 'Seafood', 'Tomato', 'Cheese', 'Salad', 'Baked Goods', 'Baked Goods', 'Cheese', 'Salad'], 'bboxes': [[0.37747705, 1.0, 0.31908837, 0.981482148], [0.0115042031, 0.553094864, 0.0954515785, 0.728246], [0.00692665577, 0.548264325, 0.204499051, 0.702045619], [0.38464877, 0.958207607, 0.193982765, 0.977637291], [0.17925939, 0.906846285, 0.173706621, 0.842920542], [0.184155583, 0.861931205, 0.132575363, 0.933382452], [0.00305101275, 0.611860037, 0.0430814438, 0.757687569], [0.339314103, 0.976260185, 0.193029925, 1.0], [0.226356596, 0.930693269, 0.581959367, 0.95653224], [0.179834038, 0.908775806, 0.0958697423, 0.939691424], [0.432378292, 0.933267236, 0.383782387, 0.998322189], [0.284259945, 0.931147337, 0.431660354, 0.955191612], [0.380976766, 0.981038809, 0.240252972, 0.948227406], [0.0822101831, 0.80000484, 0.0997514725, 0.613626182], [0.0185090601, 0.71911788, 0.238465369, 0.786021173], [0.226215214, 0.90732789, 0.745254755, 0.976904333], [0.0948396623, 0.77132678, 0.0453846678, 0.7058779], [0.0, 0.600441575, 0.0679637417, 0.726050138], [0.357916325, 0.954997301, 0.135904908, 0.931971073], [0.132627249, 0.893175125, 0.0913171396, 0.866321087], [0.0602070391, 0.563761592, 0.0754725188, 0.508511], [0.0, 0.604042649, 0.0415849313, 0.614845335], [0.229078263, 0.878014445, 0.402972162, 0.969400823], [0.414384753, 0.949710369, 0.0264594965, 0.378633976], [0.241480172, 0.879761755, 0.151504755, 0.885163963], [0.0922716856, 0.839965582, 0.0569753647, 0.344792545], [0.421173453, 0.909894347, 0.0263550486, 0.424664497], [0.469087541, 0.929275, 0.0197226219, 0.430724025], [0.0161168873, 0.577470899, 0.803146243, 0.980094135], [0.274543285, 1.0, 0.508762538, 0.98746717], [0.254469335, 0.936549246, 0.0800963044, 0.473506063], [0.115917385, 0.850970387, 0.0382353887, 0.610701382], [0.215452224, 0.915474534, 0.672911525, 0.974969745], [0.339072973, 0.927099466, 0.545033097, 0.961636961], [0.0, 0.521227062, 0.804131687, 0.996689677], [0.476845741, 0.914084792, 0.00162035227, 0.416255236], [0.00672900677, 0.79107517, 0.23690711, 0.807291448], [0.0806772411, 0.963176131, 0.715660572, 0.979084849], [0.029843241, 0.80046916, 0.0362186842, 0.465307206], [0.375386357, 0.906366706, 0.00989544392, 0.302769661]]}], deployed_model_id='2225068486990757888', explanations=None)

## Make online predictions

### [predictions.deploy-model-api](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

## Deploy the model

Next, deploy your model for online prediction. To deploy the model, you invoke the `deploy` method.

In [ ]:
endpoint = model.deploy()

*Example output:*

    INFO:google.cloud.aiplatform.models:Creating Endpoint
    INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/4087251132693348352
    INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
    INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/759209241365/locations/us-central1/endpoints/4867177336350441472')
    INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/1691336130932244480
    INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/759209241365/locations/us-central1/endpoints/4867177336350441472

### [predictions.online-prediction-automl](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

### Get test item

You will use an arbitrary example out of the dataset as a test item. Don't be concerned that the example was likely used in training the model -- we just want to demonstrate how to make a prediction.

In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n1
cols = str(test_items[0]).split(",")
if len(cols) == 11:
    test_item = str(cols[1])
    test_label = str(cols[2])
else:
    test_item = str(cols[0])
    test_label = str(cols[1])

print(test_item, test_label)

### Make the prediction

Now that your `Model` resource is deployed to an `Endpoint` resource, you can do online predictions by sending prediction requests to the Endpoint resource.

#### Request

Since in this example your test item is in a Cloud Storage bucket, you open and read the contents of the image using `tf.io.gfile.Gfile()`. To pass the test data to the prediction service, you encode the bytes into base64 -- which makes the content safe from modification while transmitting binary data over the network.

The format of each instance is:

    { 'content': { 'b64': base64_encoded_bytes } }

Since the `predict()` method can take multiple items (instances), send your single test item as a list of one test item.

#### Response

The response from the `predict()` call is a Python dictionary with the following entries:

- `ids`: The internal assigned unique identifiers for each prediction request.
- `displayNames`: The class names for each class label.
- `confidences`: The predicted confidence, between 0 and 1, per class label.
- `bboxes`: The bounding box of each detected object.
- `deployed_model_id`: The Vertex AI identifier for the deployed Model resource which did the predictions.

In [ ]:
import base64

import tensorflow as tf

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)

*Example output:*

    Prediction(predictions=[{'ids': ['2250776168359788544', '1097854663752941568', '2250776168359788544', '1097854663752941568', '1097854663752941568', '2250776168359788544', '5709540682180329472', '1097854663752941568', '2250776168359788544', '2250776168359788544', '3403697672966635520', '2250776168359788544', '3403697672966635520', '5709540682180329472', '8015383691394023424', '5709540682180329472', '3403697672966635520', '3403697672966635520', '3403697672966635520', '1097854663752941568', '2250776168359788544', '2250776168359788544', '1097854663752941568', '2250776168359788544', '2250776168359788544', '3403697672966635520', '8015383691394023424', '1097854663752941568', '2250776168359788544', '1097854663752941568', '2250776168359788544', '2250776168359788544', '5709540682180329472', '1097854663752941568', '1097854663752941568', '2250776168359788544', '5709540682180329472', '8015383691394023424', '2250776168359788544', '3403697672966635520'], 'displayNames': ['Salad', 'Baked Goods', 'Salad', 'Baked Goods', 'Baked Goods', 'Salad', 'Seafood', 'Baked Goods', 'Salad', 'Salad', 'Cheese', 'Salad', 'Cheese', 'Seafood', 'Tomato', 'Seafood', 'Cheese', 'Cheese', 'Cheese', 'Baked Goods', 'Salad', 'Salad', 'Baked Goods', 'Salad', 'Salad', 'Cheese', 'Tomato', 'Baked Goods', 'Salad', 'Baked Goods', 'Salad', 'Salad', 'Seafood', 'Baked Goods', 'Baked Goods', 'Salad', 'Seafood', 'Tomato', 'Salad', 'Cheese'], 'bboxes': [[0.423218071, 0.979368508, 0.339486301, 0.953063667], [0.00295934081, 0.562559605, 0.0866475701, 0.733601213], [0.446993053, 0.985957086, 0.103116274, 1.0], [0.0, 0.798019052, 0.0843312368, 0.910729289], [0.200755239, 0.878952861, 0.161852047, 0.962031841], [0.141423374, 0.893875718, 0.0966037139, 0.975178], [0.446385473, 0.954654098, 0.322997868, 0.955895185], [0.0540368557, 0.735850155, 0.22394672, 0.783547163], [0.0511586964, 0.590276837, 0.247575879, 0.904480815], [0.291286081, 0.943224788, 0.742888927, 0.981343627], [0.0209743679, 0.534335613, 0.145708829, 0.751705289], [0.286338538, 0.956577778, 0.584680378, 0.96791], [0.493991911, 0.92484349, 0.385724247, 0.975459754], [0.320449412, 0.9483518, 0.16373083, 1.0], [0.458497226, 0.937700331, 0.342862546, 0.935759783], [0.0386219025, 0.711036801, 0.0140583916, 0.901848], [0.484096915, 0.925073504, 0.0282530189, 0.426966846], [0.329300791, 0.922743678, 0.158341929, 1.0], [0.472070664, 0.945299745, 0.0491278172, 0.907515883], [0.395479739, 0.947133362, 0.233833641, 0.983137965], [0.498143733, 0.962923825, 0.000415623188, 0.462042093], [0.168997586, 0.846541345, 0.472939, 0.953329563], [0.0585409701, 0.603083611, 0.0806979761, 0.442619056], [0.149338543, 0.87841326, 0.68745774, 0.975087523], [0.353095531, 0.95818007, 0.0186360981, 0.329624653], [0.127367049, 0.796614647, 0.0658644438, 0.776186824], [0.103480637, 0.728991389, 0.102527976, 0.843147755], [0.0629016757, 0.891313374, 0.0636595339, 0.424591154], [0.0277965665, 0.718145788, 0.0, 0.728123844], [0.431717843, 0.966227055, 0.0133913755, 0.449536532], [0.220863849, 0.897798777, 0.0167301502, 0.446346819], [0.128112122, 0.62433213, 0.576314211, 0.960604429], [0.0, 0.589858651, 0.0557853393, 0.740757704], [0.0763699561, 0.564970851, 0.426471323, 0.898873031], [0.0173879862, 0.97320509, 0.0287637822, 0.322233558], [0.0586977303, 0.64173305, 0.797896147, 0.986974], [0.474722087, 0.961244881, 0.00339941191, 0.444934338], [0.0472798944, 0.610487342, 0.0866674632, 0.682843804], [0.477720022, 0.945115, 0.00805068, 0.81928432], [0.223478109, 0.819148779, 0.297299504, 0.964423835]], 'confidences': [0.99927026, 0.998335898, 0.122888528, 0.0565881766, 0.0334013812, 0.0227512382, 0.019394394, 0.0163640734, 0.0106114028, 0.0095577, 0.0092699714, 0.00591981411, 0.00551535888, 0.00492637418, 0.00376707385, 0.00370016089, 0.00318370084, 0.00264257635, 0.00248732581, 0.00241615647, 0.0022392564, 0.00208666641, 0.00198837952, 0.00168023549, 0.00166417053, 0.0011962672, 0.00100666645, 0.000980117242, 0.000788055069, 0.000682824815, 0.000606221438, 0.000559259, 0.000542622234, 0.000534898543, 0.000527789292, 0.000484947581, 0.000421707897, 0.000392853981, 0.000352483446, 0.0003487629]}], deployed_model_id='5770527293638180864', explanations=None)

## Undeploy the model

When you are done doing predictions, you undeploy the model from the `Endpoint` resouce. This deprovisions all compute resources and ends billing for the deployed model.

In [ ]:
endpoint.undeploy_all()

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Pipeline
- Model
- Endpoint
- AutoML Training Job
- Batch Job
- Custom Job
- Hyperparameter Tuning Job
- Cloud Storage Bucket

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME